### The goal of this notebook is to reproduce Section 2 in Zhang et al. (2020)

This notebook attempts to reproduce the results from [Zhang et al. 2020](https://www.mdpi.com/2311-5521/5/1/2) using a two-layer QG system. Once the flow is seeded with Lagragian particles, we used the Lagrangian Averaged Vorticity Deviation (LAVD) method to identify coherent eddies.  

`pyqg` is used to spinup a two-layer QG turbulence system. We will use the same nondimensional bottom friction and beta parameters to simulate the dynamics of the Southern Ocean. 

`floater` will be helpful to implement the  Lagrangian Averaged Vorticity Deviation (LAVD) technique to detect coherent eddies. 

In [1]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import pyqg

ImportError: No module named pyqg